In [17]:
from datetime import datetime, timezone
from fetch_orders import fetch

from_ts = int(datetime(2025, 12, 28, 0, 0, tzinfo=timezone.utc).timestamp()) * 1000
await fetch(from_ts)

🔌 connected
🔐 application authenticated
🔐 account authenticated
✅ account authenticated – ready
📁 written to orders.json
❌ connection closed


In [32]:
import pandas as pd
import numpy as np
from IPython.display import display

df_orders = pd.read_json('data/orders.json')

PIP_MULTIPLIER = 10000
PIP_DIVISOR = 1000

# Helper to extract timestamps safely
def get_timestamp(x):
    return x.get('openTimestamp') if isinstance(x, dict) else np.nan

df_orders['openTimestamp_dt'] = pd.to_datetime(df_orders['tradeData'].apply(get_timestamp), unit='ms')

# --- 2. Identify Closed Positions ---
df_closing_candidates = df_orders[
    (df_orders['orderType'] == 4) &
    (df_orders['closingOrder'] == True) &
    (df_orders['executedVolume'] > 0)
]
closed_position_ids = df_closing_candidates['positionId'].unique()

position_data = []

for pos_id in closed_position_ids:
    # Get all orders for this specific position
    pos_orders = df_orders[df_orders['positionId'] == pos_id].sort_values('openTimestamp_dt')

    # 3.1 Get opening orders (Basically anything that ISN'T the closing order)
    opening_orders = pos_orders[pos_orders['closingOrder'] != True]

    if opening_orders.empty:
        # Fallback: if there's only one order and it's marked closing, 
        # use the first available order in the sequence
        opening_orders = pos_orders.head(1)

    # Calculate Total Volume Executed for this position
    total_executed_volume = opening_orders['executedVolume'].sum() / (PIP_MULTIPLIER * PIP_DIVISOR)

    # The true "Entry Price" is from the earliest opening order
    first_opener = opening_orders.iloc[0]
    entry_price = first_opener['executionPrice']
    trade_side = first_opener['tradeData']['tradeSide']

    # 3.2 Find the specific order that defines the Risk (has stopLoss)
    # We look through all opening orders for this field
    risk_order = opening_orders[opening_orders['stopLoss'].notna()]
    if not risk_order.empty:
        max_loss_pips_risked = abs((risk_order.iloc[0]['stopLoss'] - risk_order.iloc[0]['stopPrice'])) * 100
    else:
        max_loss_pips_risked = 0.0 # Or a default value if not found

    # 3.3 Get the successful closing order
    closing_order = pos_orders[
        (pos_orders['orderType'] == 4) &
        (pos_orders['closingOrder'] == True) &
        (pos_orders['executedVolume'] > 0)
    ].iloc[-1]

    exit_price = closing_order['stopPrice']

    # 3.4 Calculate Gross Pips Gained
    price_diff = exit_price - entry_price
    # If Side 2 (Short), profit is (Entry - Exit). If Side 1 (Long), profit is (Exit - Entry).
    multiplier = 1 if trade_side == 1 else -1
    gross_pips = (price_diff * PIP_MULTIPLIER / 100.0) * multiplier

    position_data.append({
        'OrderId': closing_order['orderId'],
        'Open': first_opener['openTimestamp_dt'],
        'Close': pd.to_datetime(closing_order['tradeData']['closeTimestamp'], unit='ms'),
        'Trade Side': 'Buy' if trade_side == 1 else 'Sell',
        'Volume': total_executed_volume,
        'Pips Gained': gross_pips,
        'Pips Risked': max_loss_pips_risked
    })

# --- 4. Final Tables ---
if position_data:
    position_analysis = pd.DataFrame(position_data)
    position_analysis['R:R'] = position_analysis['Pips Gained'] / position_analysis['Pips Risked']

    display(position_analysis.set_index('OrderId'))

    # Metrics Calculation
    total_trades = len(position_analysis)
    win_rate = (position_analysis['Pips Gained'] > 0).mean()
    avg_win = position_analysis[position_analysis['Pips Gained'] > 0]['Pips Gained'].mean()
    avg_loss = position_analysis[position_analysis['Pips Gained'] < 0]['Pips Gained'].mean()

    metrics_df = pd.DataFrame({
        'Metric': ['Total Trades', 'Winning Rate', 'Avg Win', 'Avg Loss', 'Expectancy'],
        'Value': [
            total_trades,
            f"{win_rate*100:.1f}%",
            f"{avg_win:.2f} pips",
            f"{abs(avg_loss):.2f} pips" if not pd.isna(avg_loss) else "0.00",
            f"{(win_rate * avg_win) - ((1-win_rate) * abs(np.nan_to_num(avg_loss))):.2f} pips"
        ]
    })
    display(metrics_df.set_index('Metric'))
else:
    print("No closed positions found with the specified criteria.")

,Open,Close,Trade Side,Volume,Pips Gained,Pips Risked,R:R
OrderId,,,,,,,
58880855,2025-12-27 18:00:43.899,2025-12-29 15:58:32.917,Buy,0.06,-68.2,60.3,-1.131012


,Value
Metric,
Total Trades,1
Winning Rate,0.0%
Avg Win,nan pips
Avg Loss,68.20 pips
Expectancy,nan pips
